In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
import operator as op
import lid_driven_cavity as lid

In [4]:
# Physics parameters
reynolds_numbers = [100, 400, 1000, 3200, 5000]

# Boundary conditions
u_top = (1, 0)
u_bot = (0, 0)
u_left = (0, 0)
u_right = (0, 0)

# Spatial parameters
Lx = 1.0
Ly = 1.0

In [13]:
import numpy as np
print(np.random.randn(2,5))

[[-0.89088888  0.94562136  1.73006796  1.05808992  0.57090536]
 [-1.26631192 -0.38703302 -0.60477799  0.96462238  0.83881705]]


In [5]:
# Physics parameter
reynolds_number = reynolds_numbers[-1]

# Spatial resolution
N = 4
h = 1.0 / N

# Temporal resolution
k = 1.0  # FIXME arbitrary for now, likely will need to decrease

# Setup

1. Assume $\phi^n$ and $u^n$ are known at all interior, boundary, and ghost points.

In [6]:
u_n = np.zeros((N**2, 2))
phi_n = np.zeros((N**2,))

2. Assume $G(\phi^n)$ is known at all interior, boundary, and ghost points.

In [ ]:
# G_phi_n = lid.gradient(phi_n)

3. Assume $N(u^{n−1})$ is known at all interior points.

In [ ]:
# N_u_nm1 = lid.convective(u_nm1)

# Prediction step

4. Compute $\hat{u}^n$ at all interior points.

In [ ]:
# uhat_n = lid.interpolate(u_n)

5. Compute $N(u^n)$ at all interior points.

In [ ]:
# N_u_n = lid.convective(u_n)

6. Evaluate $g(x, t_{n+1})$ at all required locations on the domain boundary.

In [ ]:
# g_np1 = lid.g_func(x, t_np1)

7. Solve for $u^*$ at all interior, boundary, and ghost points.

In [ ]:
# L_u_n = lid.viscous(u_n)
# ustar = lid.take_prediction_step(u_n, N_u_n, L_u_n)

# Poisson problem

8. Compute $D(u^*)$ at all interior points.

In [ ]:
# D_ustar = lid.divergence(ustar)

9. Solve for $\phi^{n+1}$ at all interior and ghost points.

In [ ]:
# phi_np1 = lid.solve_poisson(D_ustar, phi_n)

# Correction step

10. Compute $G(\phi^{n+1})$ at all interior, boundary, and ghost points.

In [ ]:
# G_phi_np1 = lid.gradient(phi_np1)

11. Correct $u^*$ to obtain $u^{n+1}$ at all interior, boundary, and ghost points.

In [ ]:
# u_np1 = lid.take_correction_step(u_star, G_phi_np1)

# Bookkeeping

12. Update counter from $n$ to $n + 1$ and time from $t_n$ to $t_{n+1}$.

In [8]:
# n = n + 1
# tn = tn + k

13. Repeat from Step 1.